<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/NLU_ASSIGNMENT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import csv
import requests
import xml.etree.ElementTree as ET

In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [7]:
tree = ET.parse("/content/gdrive/MyDrive/train.xml")
root = tree.getroot()

In [6]:
import pandas as pd
import xml.etree.ElementTree as ET
import io

def iter_docs(author):
    author_attr = author.attrib
    for doc in author.iter('text'):
        doc_dict = author_attr.copy()
        #doc_dict.update(doc.attrib)
        doc_dict['text'] = doc.text
        yield doc_dict


etree = ET.parse("/content/gdrive/MyDrive/train.xml")
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))

In [7]:
doc_df.head()

,text
0,It might be the best sit down food I've had in...
1,Hostess was extremely accommodating when we ar...
2,We were a couple of minutes late for our reser...
3,"Though the service might be a little slow, the..."
4,Although we arrived at the restaurant 10 min l...


In [ ]:
doc_df.shape

(3134, 1)

In [ ]:
for i in root.iter('')

In [64]:
import pandas as pd
import xml.etree.ElementTree as ET
import io
import xml.dom.minidom

def iter_docs(author):
    author_attr = author.attrib
    i = 0
    for doc in author.iter('sentence'):
        i += 1
        for text in doc.iter('text'):
          print(text.text)
        for j, cat in enumerate(doc.iter('aspectCategory')):
          j = j+1
          print(cat.attrib)
          print(j)
          
        print('----------')
        if i ==4:
          break
        doc_dict = author_attr.copy()
        doc_dict.update(cat.attrib)
        doc_dict['text'] = text.text
        yield doc_dict


etree = ET.parse("/content/gdrive/MyDrive/train.xml")
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))

It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.
{'category': 'food', 'polarity': 'positive'}
1
{'category': 'place', 'polarity': 'neutral'}
2
----------
Hostess was extremely accommodating when we arrived an hour early for our reservation.
{'category': 'staff', 'polarity': 'positive'}
1
{'category': 'miscellaneous', 'polarity': 'neutral'}
2
----------
We were a couple of minutes late for our reservation and minus one guest, but we didn't think we deserved the attitude we got from the hostess.
{'category': 'miscellaneous', 'polarity': 'neutral'}
1
{'category': 'staff', 'polarity': 'negative'}
2
----------
Though the service might be a little slow, the waitresses are very friendly.
{'category': 'service', 'polarity': 'negative'}
1
{'category': 'staff', 'polarity': 'positive'}
2
----------


In [65]:
doc_df.head()

,category,polarity,text
0,place,neutral,It might be the best sit down food I've had in...
1,miscellaneous,neutral,Hostess was extremely accommodating when we ar...
2,staff,negative,We were a couple of minutes late for our reser...


In [ ]:
doc_df['category'].unique()

array(['food', 'place', 'staff', 'miscellaneous', 'service', 'price',
       'menu', 'ambience'], dtype=object)

In [ ]:
doc_df.shape

(7056, 2)

In [ ]:
df = pd.get_dummies(doc_df['category'], columns = ['food', 'place', 'staff', 'miscellaneous', 'service', 'price', 'menu', 'ambience'])

In [ ]:
N = 2
df = df.groupby(df.index // N).sum()

In [ ]:
df.shape

(3528, 8)

In [ ]:
df.index

RangeIndex(start=0, stop=7056, step=1)

In [25]:
import pandas as pd
import xml.etree.ElementTree as ET
import io

def iter_docs(author):
    author_attr = author.attrib
    for cat, text in zip(author.iter('aspectCategory'), author.iter('text')):
        doc_dict = author_attr.copy()
        doc_dict.update(cat.attrib)
        doc_dict['text'] = text.text
        yield doc_dict
        


etree = ET.parse("/content/gdrive/MyDrive/train.xml")
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))

TypeError: ignored

In [24]:
doc_df.head()

,category,polarity,text
0,food,positive,It might be the best sit down food I've had in...
1,place,neutral,Hostess was extremely accommodating when we ar...
2,staff,positive,We were a couple of minutes late for our reser...
3,miscellaneous,neutral,"Though the service might be a little slow, the..."
4,miscellaneous,neutral,Although we arrived at the restaurant 10 min l...


In [ ]:
import xml.etree.ElementTree as ET
from collections import Counter

def count_tags(filename):
        my_tags = []
        for event, element in ET.iterparse(filename):
            my_tags.append(element.tag)
        my_keys = Counter(my_tags).keys()
        my_values = Counter(my_tags).values()
        my_dict = dict(zip(my_keys, my_values))
        return my_dict

print(count_tags("/content/gdrive/MyDrive/train.xml"))

{'text': 3134, 'aspectCategory': 7056, 'aspectCategories': 3134, 'sentence': 3134, 'sentences': 1}


In [ ]:
from xml.dom.minidom import parseString
file = open("/content/gdrive/MyDrive/train.xml",'r')
data = file.read()
file.close()
dom = parseString(data)
print(len(dom.getElementsByTagName('text')))

3134


In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import io

def iter_docs(author):
    author_attr = author.attrib
    for cat in author.iter('aspectCategory'):
      for text in author.iter('text'):
        item = (text.text, cat.attrib)
        #print(text.text == text.text)
        print(item)
        doc_dict = author_attr.copy()
        doc_dict.update(cat.attrib)
        doc_dict['text'] = text.text
        break
        yield doc_dict
        


etree = ET.parse("/content/gdrive/MyDrive/train.xml")
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))

Streaming output truncated to the last 5000 lines.
("It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.", {'category': 'service', 'polarity': 'negative'})
("It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.", {'category': 'miscellaneous', 'polarity': 'neutral'})
("It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.", {'category': 'staff', 'polarity': 'negative'})
("It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.", {'category': 'place', 'polarity': 'negative'})
("It might be the best sit down food I've had in the area, so if you are going to the upright ci

In [ ]:
pd.read_xml("/content/gdrive/MyDrive/train.xml")

,text,aspectCategories
0,It might be the best sit down food I've had in...,NaN
1,Hostess was extremely accommodating when we ar...,NaN
2,We were a couple of minutes late for our reser...,NaN
3,"Though the service might be a little slow, the...",NaN
4,Although we arrived at the restaurant 10 min l...,NaN
...,...,...
3129,"It took us 20 minutes to get a check, despite ...",NaN
3130,"The pancakes were good, but don't have blueber...",NaN
3131,"Our waiter disappeared entirely, our food, whe...",NaN
3132,90 for a porterhouse steak for 2 (ordered medi...,NaN


In [59]:
root.findall("./sentence/aspectCategories/aspectCategory[last()]")

[<Element 'aspectCategory' at 0x7fecb2ad8ad0>,
 <Element 'aspectCategory' at 0x7fecb2ad8cb0>,
 <Element 'aspectCategory' at 0x7fecb2ad8e90>,
 <Element 'aspectCategory' at 0x7fecb2afc0b0>,
 <Element 'aspectCategory' at 0x7fecb2afc290>,
 <Element 'aspectCategory' at 0x7fecb2afc4d0>,
 <Element 'aspectCategory' at 0x7fecb2afc710>,
 <Element 'aspectCategory' at 0x7fecb2afc8f0>,
 <Element 'aspectCategory' at 0x7fecb2afcad0>,
 <Element 'aspectCategory' at 0x7fecb2afccb0>,
 <Element 'aspectCategory' at 0x7fecb2afcef0>,
 <Element 'aspectCategory' at 0x7fecb2a82110>,
 <Element 'aspectCategory' at 0x7fecb2a822f0>,
 <Element 'aspectCategory' at 0x7fecb2a824d0>,
 <Element 'aspectCategory' at 0x7fecb2a826b0>,
 <Element 'aspectCategory' at 0x7fecb2a82890>,
 <Element 'aspectCategory' at 0x7fecb2a82a70>,
 <Element 'aspectCategory' at 0x7fecb2a82c50>,
 <Element 'aspectCategory' at 0x7fecb2a82e30>,
 <Element 'aspectCategory' at 0x7fecb2a860b0>,
 <Element 'aspectCategory' at 0x7fecb2a86290>,
 <Element 'as

In [58]:
j = 0
for text in root.findall('./sentence/text'):
  j +=1
  i =0
  print(text.text)
  for cat in root.findall('./sentence/aspectCategories/aspectCategory[last()]'):
    print(cat.attrib)
    i = i+1
    if i ==2:
      break
  if j ==4:
    break

It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.
{}
Hostess was extremely accommodating when we arrived an hour early for our reservation.
{}
We were a couple of minutes late for our reservation and minus one guest, but we didn't think we deserved the attitude we got from the hostess.
{}
Though the service might be a little slow, the waitresses are very friendly.
{}


In [17]:
print(root)

<Element 'sentences' at 0x7fecb2ad8770>


In [ ]:
doc_df.shape

(3134, 3)

In [17]:
from xml.dom import minidom
xmldoc = minidom.parse('/content/gdrive/MyDrive/train.xml')
itemlist = xmldoc.getElementsByTagName('aspectCategory')
print(len(itemlist))
print(itemlist[0].attributes['aspectCategories'].value)
for s in itemlist:
    print(s.attributes['name'].value)

7056


KeyError: ignored